In [1]:
def is_colab():
    """Check if the code is running in Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Use the function to conditionally run magic commands
if is_colab():
    # Run Colab-specific magic commands
    print("Running in Colab, executing magic commands.")
    !rm -rf colabtest
    !git clone https://github.com/juan-carvajal/colabtest.git
    !cp -R colabtest/* .
    # Add any other Colab-specific setup
else:
    # Alternative setup for non-Colab environments
    print("Not running in Colab, nothing else is needed.")

Not running in Colab, nothing else is needed.


In [2]:
with open('data/test.txt', 'r') as f:
  print(f.read())

holii
